# Build a supervisor agent

## Overview

The **supervisor pattern** is a [multi-agent](/oss/python/langchain/multi-agent) architecture where a central supervisor agent coordinates specialized worker agents. This approach excels when tasks require different types of expertise. Rather than building one agent that manages tool selection across domains, you create focused specialists coordinated by a supervisor who understands the overall workflow.

In this tutorial, you'll build a personal assistant system that demonstrates these benefits through a realistic workflow. The system will coordinate two specialists with fundamentally different responsibilities:

* A **calendar agent** that handles scheduling, availability checking, and event management.
* An **email agent** that manages communication, drafts messages, and sends notifications.

We will also incorporate [human-in-the-loop review](/oss/python/langchain/human-in-the-loop) to allow users to approve, edit, and reject actions (such as outbound emails) as desired.

### Why use a supervisor?

Multi-agent architectures allow you to partition [tools](/oss/python/langchain/tools) across workers, each with their own individual prompts or instructions. Consider an agent with direct access to all calendar and email APIs: it must choose from many similar tools, understand exact formats for each API, and handle multiple domains simultaneously. If performance degrades, it may be helpful to separate related tools and associated prompts into logical groups (in part to manage iterative improvements).

### Concepts

We will cover the following concepts:

* [Multi-agent systems](/oss/python/langchain/multi-agent)
* [Human-in-the-loop review](/oss/python/langchain/human-in-the-loop)

---

## 개요

**슈퍼바이저 패턴**은 중앙의 슈퍼바이저 에이전트가 전문화된 워커 에이전트들을 조율하는 [멀티 에이전트](/oss/python/langchain/multi-agent) 아키텍처입니다. 이 접근 방식은 작업이 서로 다른 유형의 전문성을 요구할 때 탁월합니다. 여러 도메인에 걸쳐 도구 선택을 관리하는 하나의 에이전트를 만드는 대신, 전체 워크플로우를 이해하는 슈퍼바이저가 조율하는 집중된 전문가들을 만듭니다.

이 튜토리얼에서는 현실적인 워크플로우를 통해 이러한 이점을 보여주는 개인 비서 시스템을 구축합니다. 이 시스템은 근본적으로 다른 책임을 가진 두 전문가를 조율합니다:

* 일정 잡기, 가용성 확인, 이벤트 관리를 담당하는 **캘린더 에이전트**
* 커뮤니케이션 관리, 메시지 작성, 알림 전송을 담당하는 **이메일 에이전트**

또한 사용자가 원하는 대로 작업(예: 발신 이메일)을 승인, 편집, 거부할 수 있도록 [휴먼인더루프 검토](/oss/python/langchain/human-in-the-loop)를 포함할 것입니다.

### 왜 슈퍼바이저를 사용하나요?

멀티 에이전트 아키텍처는 [도구](/oss/python/langchain/tools)를 워커들에게 분할하여, 각 워커가 자체 프롬프트나 지침을 가질 수 있게 합니다. 모든 캘린더와 이메일 API에 직접 접근하는 에이전트를 생각해보세요: 많은 유사한 도구 중에서 선택하고, 각 API의 정확한 형식을 이해하며, 여러 도메인을 동시에 처리해야 합니다. 성능이 저하되면, 관련 도구와 연관된 프롬프트를 논리적 그룹으로 분리하는 것이 도움이 될 수 있습니다(부분적으로 반복적인 개선을 관리하기 위해).

### 개념

다음 개념들을 다룰 것입니다:

* [멀티 에이전트 시스템](/oss/python/langchain/multi-agent)
* [휴먼인더루프 검토](/oss/python/langchain/human-in-the-loop)

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", temperature=0)

## 1. Define tools

Start by defining the tools that require structured inputs. In real applications, these would call actual APIs (Google Calendar, SendGrid, etc.). For this tutorial, you'll use stubs to demonstrate the pattern.

---

## 1. 도구 정의

구조화된 입력이 필요한 도구를 정의하는 것부터 시작합니다. 실제 애플리케이션에서는 실제 API(Google Calendar, SendGrid 등)를 호출할 것입니다. 이 튜토리얼에서는 패턴을 보여주기 위해 스텁을 사용합니다.

In [ ]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,       # ISO format: "2024-01-15T14:00:00"
    end_time: str,         # ISO format: "2024-01-15T15:00:00"
    attendees: list[str],  # email addresses
    location: str = ""
) -> str:
    """Create a calendar event. Requires exact ISO datetime format."""
    # Stub: In practice, this would call Google Calendar API, Outlook API, etc.
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"


@tool
def send_email(
    to: list[str],  # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    # Stub: In practice, this would call SendGrid, Gmail API, etc.
    return f"Email sent to {', '.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """Check calendar availability for given attendees on a specific date."""
    # Stub: In practice, this would query calendar APIs
    return ["09:00", "14:00", "16:00"]

## 2. Create specialized sub-agents

Next, we'll create specialized sub-agents that handle each domain.

### Create a calendar agent

The calendar agent understands natural language scheduling requests and translates them into precise API calls. It handles date parsing, availability checking, and event creation.

---

## 2. 전문화된 서브 에이전트 생성

다음으로, 각 도메인을 처리하는 전문화된 서브 에이전트를 생성합니다.

### 캘린더 에이전트 생성

캘린더 에이전트는 자연어 일정 요청을 이해하고 이를 정확한 API 호출로 변환합니다. 날짜 파싱, 가용성 확인, 이벤트 생성을 처리합니다.

In [ ]:
from langchain.agents import create_agent


CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
)

Test the calendar agent to see how it handles natural language scheduling:

---

캘린더 에이전트가 자연어 일정 요청을 어떻게 처리하는지 테스트해봅니다:

In [ ]:
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

for step in calendar_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

The agent parses "next Tuesday at 2pm" into ISO format ("2024-01-16T14:00:00"), calculates the end time, calls create_calendar_event, and returns a natural language confirmation.

---

에이전트는 "next Tuesday at 2pm"을 ISO 형식("2024-01-16T14:00:00")으로 파싱하고, 종료 시간을 계산하고, create_calendar_event를 호출하고, 자연어 확인을 반환합니다.

### Create an email agent

The email agent handles message composition and sending. It focuses on extracting recipient information, crafting appropriate subject lines and body text, and managing email communication.

---

### 이메일 에이전트 생성

이메일 에이전트는 메시지 작성과 전송을 처리합니다. 수신자 정보 추출, 적절한 제목과 본문 작성, 이메일 커뮤니케이션 관리에 집중합니다.

In [ ]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

Test the email agent with a natural language request:

---

자연어 요청으로 이메일 에이전트를 테스트합니다:

In [ ]:
query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

The agent infers the recipient from the informal request, crafts a professional subject line and body, calls send_email, and returns a confirmation. Each sub-agent has a narrow focus with domain-specific tools and prompts, allowing it to excel at its specific task.

---

에이전트는 비공식적인 요청에서 수신자를 추론하고, 전문적인 제목과 본문을 작성하고, send_email을 호출하고, 확인을 반환합니다. 각 서브 에이전트는 도메인별 도구와 프롬프트로 좁은 범위에 집중하여 특정 작업에서 탁월한 성능을 발휘합니다.

## 3. Wrap sub-agents as tools

Now wrap each sub-agent as a tool that the supervisor can invoke. This is the key architectural step that creates the layered system. The supervisor will see high-level tools like "schedule\_event", not low-level tools like "create\_calendar\_event".

---

## 3. 서브 에이전트를 도구로 래핑

이제 각 서브 에이전트를 슈퍼바이저가 호출할 수 있는 도구로 래핑합니다. 이것이 계층화된 시스템을 만드는 핵심 아키텍처 단계입니다. 슈퍼바이저는 "create\_calendar\_event" 같은 저수준 도구가 아닌 "schedule\_event" 같은 고수준 도구를 보게 됩니다.

In [ ]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

The tool descriptions help the supervisor decide when to use each tool, so make them clear and specific. We return only the sub-agent's final response, as the supervisor doesn't need to see intermediate reasoning or tool calls.

---

도구 설명은 슈퍼바이저가 각 도구를 언제 사용할지 결정하는 데 도움이 되므로, 명확하고 구체적으로 작성합니다. 슈퍼바이저가 중간 추론이나 도구 호출을 볼 필요가 없으므로, 서브 에이전트의 최종 응답만 반환합니다.

## 4. Create the supervisor agent

Now create the supervisor that orchestrates the sub-agents. The supervisor only sees high-level tools and makes routing decisions at the domain level, not the individual API level.

---

## 4. 슈퍼바이저 에이전트 생성

이제 서브 에이전트를 조율하는 슈퍼바이저를 생성합니다. 슈퍼바이저는 고수준 도구만 보고, 개별 API 수준이 아닌 도메인 수준에서 라우팅 결정을 합니다.

In [ ]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
)

## 5. Use the supervisor

Now test your complete system with complex requests that require coordination across multiple domains:

### Example 1: Simple single-domain request

---

## 5. 슈퍼바이저 사용

이제 여러 도메인에 걸친 조율이 필요한 복잡한 요청으로 완성된 시스템을 테스트합니다:

### 예제 1: 간단한 단일 도메인 요청

In [ ]:
query = "Schedule a team standup for tomorrow at 9am"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

The supervisor identifies this as a calendar task, calls schedule_event, and the calendar agent handles date parsing and event creation.

### Example 2: Complex multi-domain request

---

슈퍼바이저는 이것을 캘린더 작업으로 식별하고, schedule_event를 호출하고, 캘린더 에이전트가 날짜 파싱과 이벤트 생성을 처리합니다.

### 예제 2: 복잡한 멀티 도메인 요청

In [ ]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

The supervisor recognizes this requires both calendar and email actions, calls schedule_event for the meeting, then calls manage_email for the reminder. Each sub-agent completes its task, and the supervisor synthesizes both results into a coherent response.

---

슈퍼바이저는 이것이 캘린더와 이메일 작업 모두 필요하다는 것을 인식하고, 회의를 위해 schedule_event를 호출한 다음, 리마인더를 위해 manage_email을 호출합니다. 각 서브 에이전트가 작업을 완료하고, 슈퍼바이저가 두 결과를 일관된 응답으로 종합합니다.

### Understanding the architecture

Your system has three layers. The bottom layer contains rigid API tools that require exact formats. The middle layer contains sub-agents that accept natural language, translate it to structured API calls, and return natural language confirmations. The top layer contains the supervisor that routes to high-level capabilities and synthesizes results.

This separation of concerns provides several benefits: each layer has a focused responsibility, you can add new domains without affecting existing ones, and you can test and iterate on each layer independently.

## 6. Add human-in-the-loop review

It can be prudent to incorporate [human-in-the-loop review](/oss/python/langchain/human-in-the-loop) of sensitive actions. LangChain includes [built-in middleware](/oss/python/langchain/human-in-the-loop#configuring-interrupts) to review tool calls, in this case the tools invoked by sub-agents.

Let's add human-in-the-loop review to both sub-agents:

* We configure the `create_calendar_event` and `send_email` tools to interrupt, permitting all [response types](/oss/python/langchain/human-in-the-loop) (`approve`, `edit`, `reject`)
* We add a [checkpointer](/oss/python/langchain/short-term-memory) **only to the top-level agent**. This is required to pause and resume execution.

---

### 아키텍처 이해

시스템은 세 개의 레이어를 가집니다. 하단 레이어는 정확한 형식이 필요한 엄격한 API 도구를 포함합니다. 중간 레이어는 자연어를 받아 구조화된 API 호출로 변환하고 자연어 확인을 반환하는 서브 에이전트를 포함합니다. 상단 레이어는 고수준 기능으로 라우팅하고 결과를 종합하는 슈퍼바이저를 포함합니다.

이러한 관심사 분리는 여러 이점을 제공합니다: 각 레이어가 집중된 책임을 가지고, 기존 도메인에 영향을 주지 않고 새 도메인을 추가할 수 있으며, 각 레이어를 독립적으로 테스트하고 반복할 수 있습니다.

## 6. 휴먼인더루프 검토 추가

민감한 작업에 [휴먼인더루프 검토](/oss/python/langchain/human-in-the-loop)를 포함하는 것이 현명할 수 있습니다. LangChain은 도구 호출을 검토하기 위한 [내장 미들웨어](/oss/python/langchain/human-in-the-loop#configuring-interrupts)를 포함합니다. 이 경우 서브 에이전트가 호출하는 도구입니다.

두 서브 에이전트에 휴먼인더루프 검토를 추가해봅시다:

* `create_calendar_event`와 `send_email` 도구가 중단되도록 구성하여, 모든 [응답 유형](/oss/python/langchain/human-in-the-loop)(`approve`, `edit`, `reject`)을 허용합니다
* **최상위 에이전트에만** [체크포인터](/oss/python/langchain/short-term-memory)를 추가합니다. 실행을 일시 중지하고 재개하는 데 필요합니다.

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 


calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"create_calendar_event": True}, 
            description_prefix="Calendar event pending approval", 
        ), 
    ], 
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"send_email": True}, 
            description_prefix="Outbound email pending approval", 
        ), 
    ], 
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver(), 
)

Let's repeat the query. Note that we gather interrupt events into a list to access downstream:

---

쿼리를 다시 실행해봅시다. 나중에 접근하기 위해 인터럽트 이벤트를 리스트에 수집하는 점에 주목하세요:

In [ ]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": "6"}}

interrupts = []
for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

This time we've interrupted execution. Let's inspect the interrupt events:

---

이번에는 실행이 중단되었습니다. 인터럽트 이벤트를 살펴봅시다:

In [ ]:
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")

We can specify decisions for each interrupt by referring to its ID using a Command. Refer to the human-in-the-loop guide for additional details. For demonstration purposes, here we will accept the calendar event, but edit the subject of the outbound email:

---

Command를 사용하여 각 인터럽트의 ID를 참조해 결정을 지정할 수 있습니다. 추가 세부 사항은 휴먼인더루프 가이드를 참조하세요. 데모 목적으로, 여기서는 캘린더 이벤트를 수락하지만 발신 이메일의 제목을 편집합니다:

In [ ]:
from langgraph.types import Command 

resume = {}
for interrupt_ in interrupts:
    if interrupt_.id == "2b56f299be313ad8bc689eff02973f16":
        # Edit email
        edited_action = interrupt_.value["action_requests"][0].copy()
        edited_action["arguments"]["subject"] = "Mockups reminder"
        resume[interrupt_.id] = {
            "decisions": [{"type": "edit", "edited_action": edited_action}]
        }
    else:
        resume[interrupt_.id] = {"decisions": [{"type": "approve"}]}

interrupts = []
for step in supervisor_agent.stream(
    Command(resume=resume), 
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

The run proceeds with our input.

## 7. Advanced: Control information flow

By default, sub-agents receive only the request string from the supervisor. You might want to pass additional context, such as conversation history or user preferences.

### Pass additional conversational context to sub-agents

---

실행이 우리의 입력으로 진행됩니다.

## 7. 고급: 정보 흐름 제어

기본적으로 서브 에이전트는 슈퍼바이저로부터 요청 문자열만 받습니다. 대화 기록이나 사용자 선호도와 같은 추가 컨텍스트를 전달하고 싶을 수 있습니다.

### 서브 에이전트에 추가 대화 컨텍스트 전달

In [ ]:
from langchain.tools import tool, ToolRuntime

@tool
def schedule_event(
    request: str,
    runtime: ToolRuntime
) -> str:
    """Schedule calendar events using natural language."""
    # Customize context received by sub-agent
    original_user_message = next(
        message for message in runtime.state["messages"]
        if message.type == "human"
    )
    prompt = (
        "You are assisting with the following user inquiry:\n\n"
        f"{original_user_message.text}\n\n"
        "You are tasked with the following sub-request:\n\n"
        f"{request}"
    )
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": prompt}],
    })
    return result["messages"][-1].text

This allows sub-agents to see the full conversation context, which can be useful for resolving ambiguities like "schedule it for the same time tomorrow" (referencing a previous conversation).

### Control what supervisor receives

You can also customize what information flows back to the supervisor:

---

이를 통해 서브 에이전트가 전체 대화 컨텍스트를 볼 수 있으며, "내일 같은 시간에 일정 잡아줘"(이전 대화 참조)와 같은 모호함을 해결하는 데 유용할 수 있습니다.

### 슈퍼바이저가 받는 정보 제어

슈퍼바이저에게 다시 흐르는 정보도 커스터마이즈할 수 있습니다:

In [ ]:
import json

@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language."""
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })

    # Option 1: Return just the confirmation message
    return result["messages"][-1].text

    # Option 2: Return structured data
    # return json.dumps({
    #     "status": "success",
    #     "event_id": "evt_123",
    #     "summary": result["messages"][-1].text
    # })

**Important:** Make sure sub-agent prompts emphasize that their final message should contain all relevant information. A common failure mode is sub-agents that perform tool calls but don't include the results in their final response.

## 8. Key takeaways

The supervisor pattern creates layers of abstraction where each layer has a clear responsibility. When designing a supervisor system, start with clear domain boundaries and give each sub-agent focused tools and prompts. Write clear tool descriptions for the supervisor, test each layer independently before integration, and control information flow based on your specific needs.

<Tip>
  **When to use the supervisor pattern**

  Use the supervisor pattern when you have multiple distinct domains (calendar, email, CRM, database), each domain has multiple tools or complex logic, you want centralized workflow control, and sub-agents don't need to converse directly with users.

  For simpler cases with just a few tools, use a single agent. When agents need to have conversations with users, use [handoffs](/oss/python/langchain/multi-agent#handoffs) instead. For peer-to-peer collaboration between agents, consider other multi-agent patterns.
</Tip>

## Next steps

Learn about [handoffs](/oss/python/langchain/multi-agent#handoffs) for agent-to-agent conversations, explore [context engineering](/oss/python/langchain/context-engineering) to fine-tune information flow, read the [multi-agent overview](/oss/python/langchain/multi-agent) to compare different patterns, and use [LangSmith](https://smith.langchain.com) to debug and monitor your multi-agent system.

***

---

**중요:** 서브 에이전트 프롬프트가 최종 메시지에 모든 관련 정보를 포함해야 한다는 점을 강조하도록 하세요. 일반적인 실패 모드는 서브 에이전트가 도구 호출을 수행하지만 결과를 최종 응답에 포함하지 않는 것입니다.

## 8. 핵심 요점

슈퍼바이저 패턴은 각 레이어가 명확한 책임을 가지는 추상화 레이어를 만듭니다. 슈퍼바이저 시스템을 설계할 때, 명확한 도메인 경계로 시작하고 각 서브 에이전트에 집중된 도구와 프롬프트를 제공하세요. 슈퍼바이저를 위한 명확한 도구 설명을 작성하고, 통합 전에 각 레이어를 독립적으로 테스트하고, 특정 요구에 따라 정보 흐름을 제어하세요.

<Tip>
  **슈퍼바이저 패턴을 언제 사용할까요**

  여러 개의 구별된 도메인(캘린더, 이메일, CRM, 데이터베이스)이 있고, 각 도메인에 여러 도구나 복잡한 로직이 있고, 중앙 집중식 워크플로우 제어를 원하며, 서브 에이전트가 사용자와 직접 대화할 필요가 없을 때 슈퍼바이저 패턴을 사용하세요.

  도구가 몇 개뿐인 간단한 경우에는 단일 에이전트를 사용하세요. 에이전트가 사용자와 대화해야 할 때는 대신 [핸드오프](/oss/python/langchain/multi-agent#handoffs)를 사용하세요. 에이전트 간 피어 투 피어 협업에는 다른 멀티 에이전트 패턴을 고려하세요.
</Tip>

## 다음 단계

에이전트 간 대화를 위한 [핸드오프](/oss/python/langchain/multi-agent#handoffs)에 대해 배우고, 정보 흐름을 미세 조정하기 위한 [컨텍스트 엔지니어링](/oss/python/langchain/context-engineering)을 탐색하고, 다양한 패턴을 비교하기 위한 [멀티 에이전트 개요](/oss/python/langchain/multi-agent)를 읽고, 멀티 에이전트 시스템을 디버그하고 모니터링하기 위해 [LangSmith](https://smith.langchain.com)를 사용하세요.

***